In [4]:
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import timm
import time

In [6]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Configurações iniciais
batch_size = 64
learning_rate = 0.001
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Transformação do dataset
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # Converte para 3 canais
    transforms.Resize((128, 128)),  # Redimensiona para 224x224
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [ ]:
# Carregamento do dataset Fashion-MNIST
train_dataset = datasets.FashionMNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.FashionMNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Função de treinamento
def train(model, loader, criterion, optimizer):
    model.train()
    total_loss = 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    return total_loss / len(loader)

In [ ]:
# Função de avaliação
def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    return accuracy

In [ ]:
# Função principal para treinar e avaliar os modelos
def train_and_evaluate(model_name, pre_training):
    # Carrega o modelo do timm e ajusta para 10 classes (Fashion-MNIST)
    model = timm.create_model(model_name, pretrained=pre_training, num_classes=10).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    print(f"\nTreinando o modelo: {model_name}")
    start_time = time.time()

    for epoch in range(num_epochs):
        train_loss = train(model, train_loader, criterion, optimizer)
        test_accuracy = evaluate(model, test_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

    end_time = time.time()
    print(f"Tempo total de treinamento para {model_name}: {(end_time - start_time) / 60:.2f} minutos\n")

In [ ]:
pre_trained_model = 'tf_efficientnetv2_s.in21k'  # EfficientNetV2
#raw_model = 'efficientnetv2_s'

train_and_evaluate(pre_trained_model, pre_training=True)

print(f"FINALIZADO {pre_trained_model}\n")

#train_and_evaluate(raw_model, pre_training=False)

#print(f"FINALIZADO {raw_model}\n")